In [2]:
#B站弹幕数据分析和可视化（比较习惯用pycharm写）

In [3]:
#首先是爬取弹幕并下载

In [4]:
import re
import requests
url = input('请输入B站视频链接: ')
res = requests.get(url)
cid = re.findall(r'"cid":(.*?),', res.text)[-1]
url = f'https://comment.bilibili.com/{cid}.xml'
res = requests.get(url)
with open(f'{cid}.xml', 'wb') as f:
    f.write(res.content)

请输入B站视频链接: https://www.bilibili.com/bangumi/play/ep17617


In [5]:
#输出弹幕文件51816463.xml

In [6]:
#处理弹幕文件
#stime: 弹幕出现时间 (s)
#mode: 弹幕类型 (< 7 时为普通弹幕)
#size: 字号
#color: 文字颜色
#date: 发送时间戳
#pool: 弹幕池ID
#author: 发送者ID
#dbid: 数据库记录ID（单调递增）

In [7]:
#了解弹幕的参数后，我们就将弹幕信息保存为danmus.csv文件

In [8]:
import re
with open('51816463.xml', encoding='utf-8') as f:
    data = f.read()
comments = re.findall('<d p="(.*?)">(.*?)</d>', data)
# print(len(comments))  # 3000
danmus = [','.join(item) for item in comments]
headers = ['stime', 'mode', 'size', 'color', 'date', 'pool', 'author', 'dbid', 'text']
headers = ','.join(headers)
danmus.insert(0, headers)

with open('danmus.csv', 'w', encoding='utf_8_sig') as f:
    f.writelines([line+'\n' for line in danmus])

In [9]:
#进行数据分析，首先进行词云分析

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
import jieba

with open('danmus.csv', encoding='utf-8') as f:
    text = " ".join([line.split(',')[-1] for line in f.readlines()])

words = jieba.cut(text)
_dict = {}
for word in words:
    if len(word) >= 2:
        _dict[word] = _dict.get(word, 0)+1
items = list(_dict.items())
items.sort(key=lambda x: x[1], reverse=True)

c = (
    WordCloud()
    .add(
        "",
        items,
        word_size_range=[20, 120],
        textstyle_opts=opts.TextStyleOpts(font_family="cursive"),
    )
    .render("wordcloud.html")
)

In [10]:
#由饼状图可知：3000条弹幕中，积极弹幕超过一半，中立弹幕有百分之三十几
#当然，弹幕调侃内容居中，而且有很多梗，会对情感分析造成很大的障碍

In [11]:
#>>> from snownlp import SnowNLP
#>>> s = SnowNLP('阿伟死了') 
#>>> s.sentiments

In [12]:
#"阿伟死了"因带有"死"字，所以被判别为消极情绪。
#但实际上，它反映的确实积极情绪，形容对看到可爱的事物时的激动心情。

In [ ]:
from snownlp import SnowNLP
from pyecharts import options as opts
from pyecharts.charts import Pie

with open('danmus.csv', encoding='utf-8') as f:
    text = [line.split(',')[-1] for line in f.readlines()[1:]]

emotions = {
    'positive': 0,
    'negative': 0,
    'neutral': 0
}
for item in text:
    if SnowNLP(item).sentiments > 0.6:
        emotions['positive'] += 1
    elif SnowNLP(item).sentiments < 0.4:
        emotions['negative'] += 1
    else:
        emotions['neutral'] += 1
print(emotions)


c = (
    Pie()
    .add("", list(emotions.items()))
    .set_colors(["blue", "purple", "orange"])
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)"))
    .render("emotionAnalysis.html")
)

In [13]:
#精彩片段，用折线图进行分析

In [ ]:
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Line
from pyecharts.charts import Line, Grid
import pyecharts.options as opts


with open('danmus.csv', encoding='utf-8') as f:
    text = [float(line.split(',')[0]) for line in f.readlines()[1:]]


text = sorted([int(item) for item in text])
data = {}
for item in text:
    item = int(item/60)
    data[item] = data.get(item, 0)+1


x_data = list(data.keys())
y_data = list(data.values())
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#c86589'}, {offset: 1, color: '#06a7ff'}], false)"
)
area_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: '#eb64fb'}, {offset: 1, color: '#3fbbff0d'}], false)"
)
c = (
    Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js)))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="弹幕数量",
        y_axis=y_data,
        is_smooth=True,
        symbol="circle",
        symbol_size=6,
        linestyle_opts=opts.LineStyleOpts(color="#fff"),
        label_opts=opts.LabelOpts(is_show=True, position="top", color="white"),
        itemstyle_opts=opts.ItemStyleOpts(
            color="red", border_color="#fff", border_width=3
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
        areastyle_opts=opts.AreaStyleOpts(
            color=JsCode(area_color_js), opacity=1),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(type_="max")])
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="",
            pos_bottom="5%",
            pos_left="center",
            title_textstyle_opts=opts.TextStyleOpts(
                color="#fff", font_size=16),
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            boundary_gap=False,
            axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63"),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
            ),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=25,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            )
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            position="left",
            axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63"),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
            ),
            axistick_opts=opts.AxisTickOpts(
                is_show=True,
                length=15,
                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
            ),
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
            ),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line")
    )
    .render("highlights.html")
)

In [14]:
#高能时刻，知道番剧的名场面出自几分几秒，即高能时刻。

In [15]:
import re

with open('danmus.csv', encoding='utf-8') as f:
    danmus = []
    for line in f.readlines()[1:]:
        time = int(float(line.split(',')[0]))
        text = line.split(',')[-1].replace('\n', '')
        danmus.append([time, text])

danmus.sort(key=lambda x: x[0])
dict1 = {}
dict2 = {}
control = True
for item in danmus:
    if re.search('名场面(:|：)', item[1]):
        print(f'{int(item[0]/60)}m{item[0]%60}s {item[1]}')
        control = False
        break
    if '名场面' in item[1]:
        minute = int(item[0]/60)
        second = item[0] % 60
        dict1[minute] = dict1.get(minute, 0)+1
        dict2[minute] = dict2.get(minute, 0)+second
    else:
        pass
if control:
    minute= max(dict1, key=dict1.get)
    second = round(dict2[minute]/dict1[minute])
    print(f'{minute}m{second}s 名场面')

9m29s 名场面:怀中抱妹鲨


In [16]:
#福利情节，字体颜色为黄色，也就是10进制颜色的值为16776960时，就是那种比较污的福利情节
#同时为了防止异常，只有当该分钟内出现黄色弹幕的次数≥3时，才说明该分钟内是福利情节，并且输出该分钟内第一次出现黄色弹幕的秒数：

In [ ]:
with open('danmus.csv', encoding='utf-8') as f:
    danmus = []
    for line in f.readlines()[1:]:
        time = int(float(line.split(',')[0]))
        color = line.split(',')[3]
        text = line.split(',')[-1].replace('\n', '')
        danmus.append([time, color, text])

danmus.sort(key=lambda x: x[0])
dict1 = {}
dict2 = {}
for item in danmus:
    if item[1] == '16776960':
        minute = int(item[0]/60)
        second = item[0] % 60
        dict1[minute] = dict1.get(minute, 0)+1
        if dict2.get(minute) == None:
            dict2[minute] = f'{minute:0>2}m{second:0>2}s {item[2]}'
        else:
            pass
            
for key, value in dict1.items():
    if value >= 3:
        print(dict2[key])

In [17]:
#end